# Import Libraries

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import drive

# Import dataset

In [39]:
# drive way
file_path = '/content/drive/MyDrive/iris/iris.data'

# Read CSV (or .data) file
df = pd.read_csv(file_path, header=None)

# Features (X) and target (y)
X = df.iloc[:, :-1].values  # First 4 columns
y, _ = pd.factorize(df.iloc[:, -1])  # last column

# standardize data
scaler = StandardScaler()
x = scaler.fit_transform(X)

# train test split into two
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

def to_tensor(data, target):
    return torch.tensor(data, dtype=torch.float32), torch.tensor(target, dtype=torch.long)

X_train, y_train = to_tensor(X_train, y_train)
X_test, y_test = to_tensor(X_test, y_test)

# Defining the RBFN model and rbf_kernel

In [40]:
def rbf_kernel(X, centers, beta):
    return torch.exp(-beta * torch.cdist(X, centers) ** 2)


class RBFN(nn.Module):
    def __init__(self, input_dim, num_centers, output_dim):
        super(RBFN, self).__init__()
        nn.Parameter(torch.randn(num_centers, input_dim))
        self.centers = nn.Parameter(torch.randn(num_centers, input_dim)) # randomly initialize rbf centers
        self.beta = nn.Parameter(torch.ones(1) * 2.0) # The beta parameter will control the width of the RBF.
        self.linear = nn.Linear(num_centers, output_dim) # direct output to fully connected layer


    def forward(self, x): # forward propagation
        # Calculate the rbf kernel function
        phi = rbf_kernel(x, self.centers, self.beta)
        return self.linear(phi)

# model = RBFN(4, 10, 3)

# Model training

In [41]:
num_centers = 10
model = RBFN(input_dim=4, num_centers=num_centers, output_dim=3)

# loss function definition and optimization
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# let's train the model
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad() # gradients to zero
    outputs = model(X_train) # forward propagation
    loss = criterion(outputs, y_train) # calculate loss
    loss.backward()
    optimizer.step() # update weights

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 1.0689
Epoch [20/100], Loss: 1.0206
Epoch [30/100], Loss: 0.9590
Epoch [40/100], Loss: 0.8765
Epoch [50/100], Loss: 0.7757
Epoch [60/100], Loss: 0.6562
Epoch [70/100], Loss: 0.5269
Epoch [80/100], Loss: 0.4064
Epoch [90/100], Loss: 0.3132
Epoch [100/100], Loss: 0.2490


# Test and evaluation

In [44]:
with torch.no_grad():
    y_pred = model(X_test) # predict with test data
    accuracy = (torch.argmax(y_pred, axis=1) == y_test).float().mean().item() # calculate accuracy
    print(f'Accuracy: {accuracy * 100:.2f}')

Accuracy: 97.78
